# CKME136X10 2018 Initial Data Cleaning

In [1]:
#import requried libraries
import pandas as pd
import numpy as np
import random as rand

from pandas.api.types import CategoricalDtype

pd.options.display.max_rows = 100
pd.options.display.max_columns=25

inputfile = 'NCDB_1999_to_2016.csv'
outputfile = 'CKME136X10_2018_Initial_Data_Cleaned.csv'

## Read the Data

In [2]:
df = pd.read_csv(inputfile, engine = 'python')

In [3]:
## inspect the imported data

In [4]:
print('Display the first few rows of the data')
display(df.head())
print()
print('Number of Rows: {}'.format(df.shape[0]))
print('Number of Columns: {}'.format(df.shape[1]))
print()
print('Column Names: {}'.format(df.columns.values))
print()
print('Information on the imported data')
print(df.info())

Display the first few rows of the data


,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,C_CASE
0,1999,01,1,20,2,02,34,UU,1,5,3,03,01,06,1990,01,M,41,11,1,UU,1,752
1,1999,01,1,20,2,02,34,UU,1,5,3,03,02,01,1987,01,M,19,11,1,UU,1,752
2,1999,01,1,20,2,02,34,UU,1,5,3,03,02,01,1987,02,F,20,13,2,02,2,752
3,1999,01,1,08,2,01,01,UU,5,3,6,18,01,01,1986,01,M,46,11,1,UU,1,753
4,1999,01,1,08,2,01,01,UU,5,3,6,18,99,NN,NNNN,01,M,05,99,2,UU,3,753



Number of Rows: 6486831
Number of Columns: 23

Column Names: ['C_YEAR' 'C_MNTH' 'C_WDAY' 'C_HOUR' 'C_SEV' 'C_VEHS' 'C_CONF' 'C_RCFG'
 'C_WTHR' 'C_RSUR' 'C_RALN' 'C_TRAF' 'V_ID' 'V_TYPE' 'V_YEAR' 'P_ID'
 'P_SEX' 'P_AGE' 'P_PSN' 'P_ISEV' 'P_SAFE' 'P_USER' 'C_CASE']

Information on the imported data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6486831 entries, 0 to 6486830
Data columns (total 23 columns):
C_YEAR    int64
C_MNTH    object
C_WDAY    object
C_HOUR    object
C_SEV     int64
C_VEHS    object
C_CONF    object
C_RCFG    object
C_WTHR    object
C_RSUR    object
C_RALN    object
C_TRAF    object
V_ID      object
V_TYPE    object
V_YEAR    object
P_ID      object
P_SEX     object
P_AGE     object
P_PSN     object
P_ISEV    object
P_SAFE    object
P_USER    object
C_CASE    int64
dtypes: int64(3), object(20)
memory usage: 1.1+ GB
None


## Identify and drop variables which will not add values to our analysis

In [5]:
#drop Collition Serverity
#For future prediction, this value will not be available. Thus remove it.
df.drop(columns = ['C_SEV'], inplace = True)

In [6]:
#drop Collition Case number
#For future prediction, this value will not be available. Thus remove it.
df.drop(columns = ['C_CASE'], inplace = True)

In [7]:
#drop Vehicle Sequence Number
#We have no means to determine how the order of the vehicles were determined.
df.drop(columns = ['V_ID'], inplace = True)

In [8]:
#drop Person Sequence Number
#We have no means to determine how the sequence number was identified.
df.drop(columns = ['P_ID'], inplace = True)

In [ ]:
# keeping V_YEAR, it will be dropped after EDA, as it should not add value for future prediction

## Move Dependent variable to the end of the dataset

In [9]:
#Move the dependent variable to the outside and drop C_CASE column
P_ISEV = df['P_ISEV']
df.drop(columns = ['P_ISEV'], inplace = True)
df['P_ISEV'] = P_ISEV

## check the number of missing values for each column

In [10]:
# note, in our dataset, missing values will be non numeric
print('Column name and the number of missing values')

df[df.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum()

Column name and the number of missing values


C_YEAR          0
C_MNTH        403
C_WDAY       1342
C_HOUR      64175
C_VEHS        541
C_CONF     516941
C_RCFG     697843
C_WTHR     111905
C_RSUR     267289
C_RALN     495711
C_TRAF     342634
V_TYPE     316543
V_YEAR     641842
P_SEX     6486831
P_AGE      440815
P_PSN      128099
P_SAFE    1371966
P_USER     210115
P_ISEV     417421
dtype: int64

## Identify non human entries and remove them from the dataset

In [11]:
# Person Injury Severity is what we are predicting.  Lets treat the missing values for this column first
# lets remove all missing values from this column, as this is what we are trying to predict.
to_drop = df['P_ISEV'].isin(['N', 'U', 'X'])
df.drop(df.index[to_drop], inplace = True)
df = df.reset_index(drop=True)
df['P_ISEV'].isin(['N', 'U', 'X']).sum()

0

In [12]:
# P_SEX, we are concerned with injuries to people.  Assuming all human beings have been classified as
# male or female, we will remove all rows which does not match male or female.
to_drop = df['P_SEX'].isin(['N', 'U', 'X'])
df.drop(df.index[to_drop], inplace = True)
df = df.reset_index(drop=True)
df['P_SEX'].isin(['N', 'U', 'X']).sum()

0

In [13]:
# P_AGE, Lets assume anyone without age assigned is non-human and remove it from the dataset
to_drop = df['P_AGE'].isin(['NN', 'UU', 'XX'])
df.drop(df.index[to_drop], inplace = True)
df = df.reset_index(drop=True)
df['P_AGE'].isin(['NN', 'UU', 'XX']).sum()

0

In [14]:
# Person's possition P_PSN, an other good indicator to identify non humman objects
# first replace QQ to other categor (Choice is other than the preceding value), map to 100
df['P_PSN'].replace(to_replace = 'QQ', value = '100', inplace = True)
# drop the remaining missing values
to_drop = df['P_PSN'].isin(['NN', 'UU', 'XX'])
df.drop(df.index[to_drop], inplace = True)
df = df.reset_index(drop=True)
df['P_PSN'].isin(['NN', 'UU', 'XX']).sum()

0

In [15]:
# Road User Class (P_USER), an other good indicator for non human objects
to_drop = df['P_USER'].isin(['U'])
df.drop(df.index[to_drop], inplace = True)
df = df.reset_index(drop=True)
df['P_USER'].isin(['U']).sum()

0

In [1]:
# P_SAFE, safty device used provides a good indicaiton if an observation was user or dummy variable.  Lets remove the dummy variables
to_drop = df['P_SAFE'].isin(['UU', 'NN', 'XX'])
df.drop(df.index[to_drop], inplace = True)
df = df.reset_index(drop=True)
df['P_SAFE'].isin(['UU', 'NN', 'XX']).sum()

NameError: name 'df' is not defined

## Identify Missing vs Other categories

In [16]:
# C_YEAR has no non-numeric values, nothing to do here

In [17]:
# Collision Month (C_MNTH), True unknowns, handle later

In [18]:
# Collision Weekady (C_WDAY), True unknowns, handle later

In [19]:
# Collision Hour (C_HOUR), True unknowns, handle later

In [20]:
# Number of Vehicle in a collision (V_VEHS), True unknowns, handle later

In [21]:
# Collision Configuration (C_CONF)
# QQ - Choice is other than the preceding values = Mark as 51 - 'Other configuraiton'
df['C_CONF'].replace(to_replace = 'QQ', value = '51', inplace = True)

In [22]:
# Non numeric values in Collision Road Configuration C_RCFG is high, lets investigate
# QQ, Choice is other than the preceding values.  Lets add this as other configuration
# map to 13
df['C_RCFG'].replace(to_replace = 'QQ', value = '13', inplace = True)

In [23]:
# Weather condition (C_WTHR)
# Q - Choice is other than the preceding values
# Add a new category 8 for Choice is other than the preceding values
df['C_WTHR'].replace(to_replace = 'Q', value = '8', inplace = True)

In [24]:
# Road Surface Configuration C_RSUR
# Q - Choice is other than the preceding values, map to 10
df['C_RSUR'].replace(to_replace = 'Q', value = '10', inplace = True)

In [25]:
# Road Alignmetn C_RALN
# Q - Choice is other than the preceding values, map to 7
df['C_RALN'].replace(to_replace = 'Q', value = '7', inplace = True)

In [26]:
# Traffic Control (C_TRAF)
# QQ - Choice is other than the preceding values, map to 19
df['C_TRAF'].replace(to_replace = 'QQ', value = '19', inplace = True)

In [27]:
# Vehicle Type (V_TYPE)
# NN - Data element is not applicable, map to 24
# QQ - Choice is other than the preceding values, map to 25
df['V_TYPE'].replace(to_replace = 'NN', value = '24', inplace = True)
df['V_TYPE'].replace(to_replace = 'QQ', value = '25', inplace = True)

In [28]:
# Vehicle Year (V_YEAR)
# this variable needs to be handled with better care.  
# NNNN - this would be for non_vehicles (people (maybe we can impute by the person age), maybe for bikes, ... )
# lets keep this simple for now and impute the lowest value
minYear = int(df['V_YEAR'].min()) - 1
print(minYear)
df['V_YEAR'].replace(to_replace = 'NNNN', value = minYear, inplace = True)

1900


In [29]:
# P_SEX, already taken care above

In [30]:
# P_AGE, already taken care above

In [31]:
# P_PSN, already taken care above

In [32]:
# P_ISEV, already taken care above

In [33]:
# column P_SAFE - Safety Device Used contains the most missing values
# Based on the data dictionary, 
# QQ => matches non of the categories
# NN => not applicable for the varaible
# Both of these are not missing values and we will map them to values 15 and 16 respectably
df['P_SAFE'].replace(to_replace = 'QQ', value = '15', inplace = True)
df['P_SAFE'].replace(to_replace = 'NN', value = '16', inplace = True)

In [34]:
# P_USER, already taken care above

In [35]:
#map F = 0 and M = 1
df['P_SEX'].replace(to_replace = 'F', value = '0', inplace = True)
df['P_SEX'].replace(to_replace = 'M', value = '1', inplace = True)

In [36]:
# note, in our dataset, missing values will be non numeric
print('Column name and the number of missing values')

df[df.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum()

Column name and the number of missing values


C_YEAR         0
C_MNTH       257
C_WDAY      1002
C_HOUR     52732
C_VEHS       266
C_CONF    153958
C_RCFG    472938
C_WTHR     74661
C_RSUR     67277
C_RALN    421284
C_TRAF    202555
V_TYPE         0
V_YEAR    195057
P_SEX          0
P_AGE          0
P_PSN          0
P_SAFE    460784
P_USER         0
P_ISEV         0
dtype: int64

In [37]:
for col in df.columns:
    print(col)
    print(df[col].unique())

C_YEAR
[1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012
 2013 2014 2015 2016]
C_MNTH
['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' 'UU']
C_WDAY
['1' '2' '3' '4' '5' '6' '7' 'U']
C_HOUR
['20' '08' '17' '15' '14' '01' '11' '13' '19' '16' '09' '02' '18' '12'
 '10' '23' '00' '06' '07' 'UU' '05' '22' '03' '21' '04']
C_VEHS
['02' '01' '03' '04' '07' '06' '09' '05' 'UU' '13' '08' '12' '14' '10'
 '16' '26' '71' '19' '25' '11' '21' '27' '15' '35' '22' '41' '46' '31'
 '18' '56' '23' '36' '17' '20' '29' '77' '28' '38' '32' '33' '54' '72'
 '40' '44' '58' '30' '24' '34' '39' '51' '57' '43' '37' '47']
C_CONF
['34' '01' '51' '04' '31' '21' '23' '03' '02' '33' 'UU' '24' '35' '41'
 '06' '32' '36' '05' '22' '25']
C_RCFG
['UU' '13' '01' '02' '03' '05' '04' '06' '08' '07' '09' '10']
C_WTHR
['1' '5' '3' '4' '7' '2' 'U' '6' '8']
C_RSUR
['5' '3' '2' '4' '1' '6' 'U' '10' '7' '9' '8']
C_RALN
['3' '6' '1' 'U' '2' '5' '4' '7']
C_TRAF
['03' '18' '01' 'UU' '06' '10' '05' '04' '

In [38]:
# lets drop the remaining unkown values
for col in df.columns:
    df[col] = df[col].apply(lambda x: str(x) if str(x).isdigit() else np.nan)

df.dropna(inplace = True)
df = df.reset_index(drop=True)

In [39]:
df.shape

(4336558, 19)

In [40]:
# note, in our dataset, missing values will be non numeric
print('Column name and the number of missing values')

df[df.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum()

Column name and the number of missing values


C_YEAR    0
C_MNTH    0
C_WDAY    0
C_HOUR    0
C_VEHS    0
C_CONF    0
C_RCFG    0
C_WTHR    0
C_RSUR    0
C_RALN    0
C_TRAF    0
V_TYPE    0
V_YEAR    0
P_SEX     0
P_AGE     0
P_PSN     0
P_SAFE    0
P_USER    0
P_ISEV    0
dtype: int64

In [41]:
df.to_csv(outputfile, encoding='utf-8', index=False)